In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization,LSTM, Reshape, SimpleRNN
from keras.layers.recurrent import GRU

Using TensorFlow backend.


In [3]:
import os
import string
import cv2

In [4]:
characters = string.ascii_lowercase+"0123456789"

In [5]:
characters

'abcdefghijklmnopqrstuvwxyz0123456789'

In [6]:
all_images = os.listdir("samples/")

In [7]:
all_images[-1], len(all_images)

('yyn57.png', 1070)

In [8]:
def preprocess_data():
    
    inp_x = np.zeros((len(all_images), 50,200,1))
    inp_y = np.zeros((len(all_images), 5,len(characters)))
    for idx, image in enumerate(all_images):
        # ----------------prepare input container
        
        
        img = cv2.imread("samples/"+image, cv2.IMREAD_GRAYSCALE)
        #print(np.max(img))
        #plt.imshow(img)
        
        # ---------------------Scale images --------------
        
        img = img/255.0
        
        image_txt = image[:-4]
        target_oht = np.zeros((5,len(characters)))
        if(len(image_txt)<6):
            img = np.reshape(img, (50,200,1))
            inp_x[idx] = img
            
             # ------------------Define targets and code them using OneHotEncoding
            target_oht = np.zeros((5,len(characters)))
            for k, char in enumerate(image_txt):
                target_oht[k, characters.find(char)] = 1
            inp_y[idx] = target_oht
    return inp_x, inp_y
        
               
        
        
        

In [9]:
X, Y = preprocess_data()

In [10]:
X.shape

(1070, 50, 200, 1)

In [11]:
Y.shape

(1070, 5, 36)

In [12]:
train_x = X[:900]
train_y = Y[:900]
test_x = X[900:]
test_y = Y[900:]

In [13]:
model = Sequential()

In [14]:
def prepare_model():
    model.add(Conv2D(64, (3,3), input_shape = (50,200,1) ,padding = 'same', activation = 'tanh'))
    #model.add(Dropout(.5))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (1,1),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'tanh', padding = 'same',))
    #model.add(Dropout(.3))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size =(1,1),padding ='same'))
    model.add(Conv2D(64, (3,3), activation = 'tanh', padding = 'same'))
    #model.add(Dropout(.1))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (3,2),padding ='same'))
    model.add(Conv2D(32, (3,3), activation = 'tanh', padding = 'same'))
  
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2),padding ='same'))
    model.add(Conv2D(32, (3,3), activation = 'tanh', padding = 'same'))
    model.add(MaxPooling2D(pool_size = (2,1),padding ='same'))
    print(model.output_shape)
    model.add(Reshape((model.output_shape[1], model.output_shape[2]*model.output_shape[3])))
    print(model.output_shape)
    #model.add(LSTM(50, batch_input_shape=(2,5,36),stateful = True, return_sequences = True))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    model.add(SimpleRNN(50, return_sequences = True,activation = 'relu'))
    


    model.add(Dense(36, activation = "softmax"))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
   
    return model
    
    

In [15]:
model =prepare_model()

(None, 5, 50, 32)
(None, 5, 1600)


In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 200, 64)       640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 200, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 200, 64)       36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 200, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 200, 64)       36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 100, 64)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 17, 100, 32)      

In [17]:
train_x.shape
train_y.shape
test_x.shape
test_y.shape

(170, 5, 36)

In [18]:
#model = create_model()
hist = model.fit(train_x, train_y, batch_size=32, epochs=50,verbose=1, validation_data=(test_x, test_y))

Train on 900 samples, validate on 170 samples
Epoch 1/50
900/900 [==============================] - 18s 20ms/step - loss: 3.3692 - accuracy: 0.0602 - val_loss: 3.3616 - val_accuracy: 0.0776
Epoch 2/50
900/900 [==============================] - 4s 5ms/step - loss: 3.0303 - accuracy: 0.0971 - val_loss: 3.3069 - val_accuracy: 0.0576
Epoch 3/50
900/900 [==============================] - 5s 5ms/step - loss: 2.9151 - accuracy: 0.0944 - val_loss: 3.3254 - val_accuracy: 0.0788
Epoch 4/50
900/900 [==============================] - 5s 5ms/step - loss: 2.8077 - accuracy: 0.1289 - val_loss: 3.3277 - val_accuracy: 0.0776
Epoch 5/50
900/900 [==============================] - 5s 5ms/step - loss: 2.6623 - accuracy: 0.1691 - val_loss: 3.2729 - val_accuracy: 0.1141
Epoch 6/50
900/900 [==============================] - 5s 5ms/step - loss: 2.4524 - accuracy: 0.2162 - val_loss: 3.2733 - val_accuracy: 0.1353
Epoch 7/50
900/900 [==============================] - 5s 5ms/step - loss: 2.2514 - accuracy: 0.2560 

In [19]:
score= model.evaluate(test_x,test_y,verbose=1)
print('Test Loss and accuracy:', score)

170/170 [==============================] - 0s 2ms/step
Test Loss and accuracy: [4.576726464664235, 0.47294119000434875]


In [20]:
# Define function to predict captcha
def predict(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    #print("img",img.shape)
    if(img is not None):
        img = cv2.resize(img, (200,50))
    if img is not None:
        img = img / 255.0
    else:
        print("Not detected");
    res = np.array(model.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 36))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        #probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += characters[l]
    return capt#, sum(probs) / 5

In [21]:
#model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
print(predict('samples/8n5p3.png'))
print(predict('samples/f2m8n.png'))
print(predict('samples/dce8y.png'))
print(predict('samples/3eny7.png'))
print(predict('samples/npxb7.png'))
print(predict('test/captcha.png'))

8n5p3
f2m8n
dce8y
3eny7
npxb7
66gmg
